In [2]:
from pathlib import Path
import os
from Bio import SeqIO, Seq

from local.constants import WORKSPACE_ROOT

In [3]:
# assembly = WORKSPACE_ROOT/"data/sean/assembly/epi300.fna"
# orf_prediction = WORKSPACE_ROOT/"data/sean/orf_prediction"

# accepted Sean's assembly as cannon
assembly = WORKSPACE_ROOT/"data/assembly/epi300.fna"
orf_prediction = WORKSPACE_ROOT/"data/orf_prediction"
orf_prediction.mkdir(parents=True, exist_ok=True)

In [3]:
if (orf_prediction/"epi300.prodigal.faa").exists():
    print("ORF prediction already done")
else:
    os.system(f"""
        prodigal -f gff \
            -i {assembly} \
            -o {orf_prediction}/epi300.gff \
            -a {orf_prediction}/epi300.prodigal.faa \
    """)

-------------------------------------
PRODIGAL v2.6.3 [February, 2016]         
Univ of Tenn / Oak Ridge National Lab
Doug Hyatt, Loren Hauser, et al.     
-------------------------------------
Request:  Single Genome, Phase:  Training
Reading in the sequence(s) to train...4691561 bp seq created, 50.78 pct GC
Locating all potential starts and stops...243330 nodes
Looking for GC bias in different frames...frame bias scores: 1.54 0.19 1.27
Building initial set of genes to train from...done!
Creating coding model and scoring nodes...done!
Examining upstream regions and training starts...done!
-------------------------------------
Request:  Single Genome, Phase:  Gene Finding
Finding genes in sequence #1 (4691561 bp)...done!


0

In [ ]:
OUTPUT = """
-------------------------------------
PRODIGAL v2.6.3 [February, 2016]         
Univ of Tenn / Oak Ridge National Lab
Doug Hyatt, Loren Hauser, et al.     
-------------------------------------
Request:  Single Genome, Phase:  Training
Reading in the sequence(s) to train...4691561 bp seq created, 50.78 pct GC
Locating all potential starts and stops...243330 nodes
Looking for GC bias in different frames...frame bias scores: 1.54 0.19 1.27
Building initial set of genes to train from...done!
Creating coding model and scoring nodes...done!
Examining upstream regions and training starts...done!
-------------------------------------
Request:  Single Genome, Phase:  Gene Finding
Finding genes in sequence #1 (4691561 bp)...done!
"""

In [4]:
with open(orf_prediction/"epi300.faa", "w") as f:
    for e in SeqIO.parse(orf_prediction/"epi300.prodigal.faa", "fasta"):
        e.seq = Seq.Seq(str(e.seq).replace("*", ""))
        e.description = e.id
        SeqIO.write(e, f, "fasta")

In [15]:
with open(orf_prediction/"epi300.bar_headers.faa", "w") as f:
    for e in SeqIO.parse(orf_prediction/"epi300.prodigal.faa", "fasta"):
        e.seq = Seq.Seq(str(e.seq).replace("*", ""))
        cont, orf = e.id.split("_")
        n = int(orf)
        c = int(cont[1:])
        # https://www.uniprot.org/help/accession_numbers
        # ns = f"{n:04}"
        # na, nb = ns[:3], ns[3:]
        a, b =  f"db|{e.id}|e{e.id}", ""
        e.id, e.description = a, b
        SeqIO.write(e, f, "fasta")

In [13]:
with open(orf_prediction/"epi300.uniprot_headers.faa", "w") as f:
    for e in SeqIO.parse(orf_prediction/"epi300.prodigal.faa", "fasta"):
        e.seq = Seq.Seq(str(e.seq).replace("*", ""))
        cont, orf = e.id.split("_")
        n = int(orf)
        c = int(cont[1:])
        # https://www.uniprot.org/help/accession_numbers
        ns = f"{n:04}"
        na, nb = ns[:3], ns[3:]
        a, b =  f"sp|C{c}P{na}X{nb}|X{ns}_ECOLI", f"hypothetical protein {e.id}"
        e.id, e.description = a, b
        SeqIO.write(e, f, "fasta")